# VCノードの作成

---

Jupyterhubを構築するためノードをVCP SDKを用いて作成します。

## はじめに

このNotebookではJupyterHub環境を構築するためのノード作成を行います。managerノードにNFSサーバを配置する構成とします。  
本ノートブックで構築する部分を以下に図示します。

![モジュール構成](images/011/arch-011.png)

### ノード構成


作成するノードはJupyterHub, PostgreSQLなどのコンテナを実行する managerノードと、single-user Jupyter notebook serverを実行する worker ノードから構成されます。

JupyterHubではNotebookなどの資材をノード間で共有するためにNFSを利用します。このNotebookで構築するノード構成ではmanagerノードにNFSサーバの機能をもたせます。

### 事前に用意が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* VCCアクセストークン
* SSH公開鍵ペア
* VCノードに割り当てるアドレス
* NTPの設定

####  VCCアクセストークン

このNotebookではVCP SDKを用いてVCノード(クラウドの計算資源)、VCディスク(クラウドのストレージ)を作成します。VCP SDKを用いてVC Controller(VCC)の操作を行うために VCCアクセストークンが必要となります。

VCCアクセストークンがない場合はVC管理者にアクセストークンの発行を依頼してください。

#### SSH公開鍵ペア

作成するVCノードに登録するSSHの公開鍵を用意する必要があります。あらかじめ作成したものをこのNotebook環境内にアップロードするか、この環境内で公開鍵ペアの作成を行ってください。

#### VCノードに割り当てるアドレス

VCノードのネットワークインターフェースに割り当てるアドレスを用意してください。VCノードにはIPアドレス、またはMACアドレスを割り当てることができます。

MACアドレスを割り当てる場合は、作成する全てのVCノード数に対応するMACアドレスを指定する必要があります。

#### NTPの設定


IdPと連携して認証を行う場合は時刻が正しく設定されていないと正しく動作しないことがあります。時刻合わせを行うためにVCノードのNTPを設定するには事前にVCコントローラへの設定が必要となります。
VCPのリリースノート「[Release/20.10.0 -- 2.機能追加](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=32677360#id-%E3%83%AA%E3%83%AA%E3%83%BC%E3%82%B9%E3%83%8E%E3%83%BC%E3%83%88-Release/20.10.0(2020/10/30))」に記されているように、OCS運用担当者にVCコントローラへのNTP設定を依頼してください。

## VCノードに関するパラメータ

JupyterHubの構築環境となるVCノードに関するパラメータを指定します。

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
import os
import sys

home = os.path.expanduser("~")
vcpsdk_config_path = os.path.join(home, 'vcp_config')

vcp = VcpSDK(vcc_access_token, config_dir=vcpsdk_config_path)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### UnitGroup名の指定

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素

このNotebookではノードの役割に応じた2つのユニットを作成します。
* managerユニット
  - JupyterHub, auth-proxy, PostgreSQLコンテナを実行するノード
  - NFSサーバを実行する
  - Docker Swarmのmanagerノード
* workerユニット
  - JupyterHubのsingle-user Jupyter Notebook server コンテナを実行するノード
  - Docker Swarm の workerノード

2つのユニットをまとめて扱うためのUnitGroupの名前を指定します。既存のUnitGroup名を確認するために現在のUnitGroup一覧を表示します。次のセルを実行し`vcname`に表示されている名前が既存のUnitGroup名となります。

In [ ]:
vcp.df_ugroups()

このNotebookで作成するUnitGroup名を指定してください。上記の一覧に表示されている名前との重複を避けて下さい。またJupyterHubのUnitGroup名には英数字のみの値を指定してください（先頭文字に数字は指定できない）。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'
# vc_provider = 'onpremises' # オンプレミスに構築する場合

vc_provider = 

### VCノードに割り当てるリソース量の指定

VCノードに割り当てるリソース量を指定します。managerノードとworkerノードでは役割が異なるため、それぞれについて指定を行います。

#### managerノード

managerノードに割り当てるリソース量を指定します。managerノードではJupyterHub, auth-proxy, PostgreSQLコンテナなどを実行します。

##### managerノードのflavor指定

個々のリソース量を毎回指定するのは煩雑となるので、VCP SDKでは典型的な構成のパラメータセットを事前に定義しています。事前に定義したパラメータセットのことをVCP SDKでは`flavor`と呼んでいます。

リソース量を指定するためのオブジェクト`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`もプロバイダ毎に定義されています。次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [ ]:
# (例)
# manager_flavor = 'small'
# manager_flavor = 'default' # オンプレミスに構築する場合、defaultを指定する

manager_flavor = 

##### managerノードのルートボリュームサイズ

managerノードに割り当てるディスクサイズを指定します。

このNotebookで構成するノード構成では manager ノードがNFSサーバの機能を兼ねることになります。`aws`, `azure`にVCノードを作成する場合は、ルートボリュームとは別にNFS用のディスクを作成しますが、他のクラウドを利用する場合はルートボリュームをNFSにも利用します。その場合はNFSのディスク使用量を考慮してルートボリュームサイズを指定してください。

本テンプレートで構築するJupyterHubシステムでは、コンテナイメージとして約2GB、single-user notebook serverコンテナのイメージとして約4GB利用します。また、作成する課題ファイルなども保存するため、`flavor`の指定によってはコンテナを起動する際にディスク容量が不足することがあります。ディスク容量不足にならないよう、ディスクサイズ設定を行ってください。なお、指定する単位は GB です。

In [ ]:
# (例)
# manager_disk_size = 16

manager_disk_size = 

#### workerノード

workerノードに割り当てるリソース量を指定します。workerノードでは各ユーザの利用環境となる single-user Jupyter notebook serverコンテナを実行します。

##### workerノードのflavor指定

workerノードの `flavor` を次のセルに指定してください。

In [ ]:
# (例)
# worker_flavor = 'small'
# worker_flavor = 'default' # オンプレミスに構築する場合、defaultを指定する

worker_flavor = 

##### workerノードのルートボリュームサイズ

workerノードのルートボリュームサイズの指定を行います。managerノード同様、容量不足にならないよう、ディスクサイズを設定してください。

In [ ]:
# (例)
# worker_disk_size = 16

worker_disk_size = 

##### ノード数の指定

作成するworkerノードのノード数を指定してください。

In [ ]:
# (例)
# worker_nodes = 1

worker_nodes = 

### アドレスの指定

VCノードのネットワークインターフェースに割り当てるアドレスを指定します。

VCノードのネットワークインタフェースに以下のアドレスを割り当てることができます。

* IPアドレス
* MACアドレス

> MACアドレスの指定が可能なのは `vmware` のみ

アドレスの指定が必要な場合は以下の設定を行ってください。指定しない場合は、自動的に割り当てられたアドレスを使用します。

#### IPアドレスを指定する場合

VCノードにIPアドレスを割り当てる場合はこの節を実行してください。

この節を実行しない場合、VCノードには割り当て可能なアドレスが自動的に設定されます。また、managerノード、workerノードのどちらか一方のみを指定することもできます。ただしworkerノードにIPアドレスを指定する場合、全てのworkerノードに対してアドレス指定を行う必要があります。

まず指定可能なIPアドレスの範囲を確認します。次のセルを実行するとVCノードに割り当て可能なIPアドレスの範囲が表示されます。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))

次のセルで manager/worker ノードに割り当てるプライベートネットワークのIPアドレスを指定します。  
ここでは、事前に各ユーザに割り当てられたIPアドレス情報を読み込んで設定します。

In [ ]:
import os

home = os.path.expanduser("~")
with open(os.path.join(home, 'nodes.txt'), "r") as f:
    lines = f.readlines()
vc_ipaddress = lines[0].replace("\n", "")
worker_ipaddresses = [
    lines[1].replace("\n", "")
]

#### MACアドレスを指定する場合

`vc_provider` に `vmware` を指定した場合、VCノードにMACアドレスを割り当てることができます。

VCノードにMACアドレスを割り当てる場合はこの節を実行してください。

VCノードにMACアドレスを割り当てない場合、この節は実行せず、飛ばしてください。

managerノード、workerノードのどちらか一方のみを指定することもできます。ただしworkerノードにMACアドレスを指定する場合、全てのworkerノードに対してアドレス指定を行う必要があります。

次のセルで manager ノードに割り当てるMACアドレスを指定してください。

In [ ]:
# (例)
# vc_mac_address = '4a:d1:4d:ab:cf:10'

vc_mac_address = 

次のセルに workerノードに割り当てるMACアドレスを指定してください。指定するアドレス数は作成するworkerノードのノード数と一致している必要があります。

In [ ]:
# (例)
# worker_mac_addresses = [
#     '4a:d1:4d:ab:cf:11',
#     '4a:d1:4d:ab:cf:12',
#     '4a:d1:4d:ab:cf:13',
# ]

worker_mac_addresses = [
    
]

### アドレスプールの指定

VCノードで実行される Docker Engineのアドレスプールの値を指定します。

本テンプレートで構築するJupyterHubでは、コンテナを実行するのにdocker swarmを利用します。docker swarmでは ingress networkなどが自動的に作成されますが、そのネットワークアドレスが既存のネットワークアドレスと重複することを避けるために docker engine のアドレスプールの値を明示的に指定します。

次のセルでdocker engineに設定するアドレスプールの値を指定してください。既存のネットワークアドレスと重複する値を避けてください。

In [ ]:
# (例)
# docker_address_pool = '10.10.0.0/16'

docker_address_pool = '10.10.0.0/16'

### SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うためにSSH公開鍵を登録します。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

> 事前に作成した公開鍵をこの環境にアップロードするか、この環境内でSSH公開鍵の作成を行ってください。

In [ ]:
# (例)
# ssh_public_key_path = '~/.ssh/id_rsa.pub'

ssh_public_key_path = '~/.ssh/id_rsa.pub'

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_private_key_path = '~/.ssh/id_rsa'

ssh_private_key_path = '~/.ssh/id_rsa'

### オンプレミスサーバに構築する場合の追加設定

オンプレミス環境（mdx含）に構築する場合、追加で設定する必要があるパラメータを指定します。  
aws, azureに構築する場合は、以下のセルを実行する必要はありません。（実行しても何も設定されません）

In [ ]:
# VCノードを構築するVMにSSHログインする際のログインユーザ名を指定します。
# （例）
# ssh_user = 'mdxuser' ...mdxに構築する場合
ssh_user = 'mdxuser'

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookではパラメータを `group_vars/` のYAMLファイルとして保存します。

ファイルに保存する前にパラメータに関する簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/check_params.py
check_parameters(
    'ugroup_name',
    'vc_provider',
    'manager_flavor',
    'manager_disk_size',
    'worker_flavor',
    'worker_disk_size',
    'worker_nodes',
    'vc_ipaddress',
    'worker_ipaddresses',
    'vc_mac_address',
    'worker_mac_addresses',
    'docker_address_pool',
    'ssh_public_key_path',
    'ssh_private_key_path',
    params={
        'vcp': vcp,
        'opt_vars': [
            'vc_ipaddress', 'worker_ipaddresses',
            'vc_mac_address', 'worker_mac_addresses',
        ],
    },
    nb_vars=locals(),
)

if vc_provider == 'onpremises':
    if not ssh_user:
        raise NotImplementedError
        print('ssh_user is required if construct on onpremises')
    
    worker_flavor = 'default'
    manager_flavor = 'default'

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
import os
%run scripts/group.py
update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
    vc_provider=vc_provider,
    manager_flavor=manager_flavor,
    manager_disk_size=manager_disk_size,
    worker_flavor=worker_flavor,
    worker_disk_size=worker_disk_size,
    worker_nodes=worker_nodes,
    docker_address_pool=docker_address_pool,
    ssh_public_key_path=os.path.expanduser(ssh_public_key_path),
    ssh_private_key_path=os.path.expanduser(ssh_private_key_path),
)
if 'vc_ipaddress' in vars():
    update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
if 'worker_ipaddresses' in vars():
    update_group_vars(ugroup_name, worker_ipaddresses=worker_ipaddresses)
if 'vc_mac_address' in vars():
    update_group_vars(ugroup_name, vc_mac_address=vc_mac_address)
if 'worker_mac_addresses' in vars():
    update_group_vars(ugroup_name, worker_mac_addresses=worker_mac_addresses)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

## VCディスクに関するパラメータ

JupyterHubに関するデータやNotebook、ユーザのホームディレクトリなどに利用するVCディスクに関するパラメータを指定します。

ディスクサイズ(GB)を指定してください。

> VCディスクを作成しないで全てのデータをルートボリュームに配置する場合は、このセクションをスキップするか、`vc_nfs_disk_size` を `0` に設定してください。
> またVCディスクに対応しているクラウドプロバイダ(aws, azure)以外を指定した場合は、ディスクサイズの指定は無効となります。

In [ ]:
# (例)
# vc_nfs_disk_size = 32

vc_nfs_disk_size = 0

VCディスクのためのUnitGroup名を指定してください。ここではVCノードのUnitGroup名から自動的に導き出した名前を設定します。

In [ ]:
disk_unit_group = ugroup_name + '_disk'
print(disk_unit_group)

この章で指定したパラメータの値をファイルに保存します。

In [ ]:
%run scripts/group.py

if 'vc_nfs_disk_size' in vars() and vc_nfs_disk_size > 0:
    if vc_provider in ['aws', 'azure']:
        update_group_vars(
            ugroup_name,
            disk_unit_group=disk_unit_group,
            vc_nfs_disk_size=vc_nfs_disk_size,
        )
    else:
        del(vc_nfs_disk_size)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

## VCディスクの作成

NFS用のVCディスクを作成します。

> 前のセクションをスキップしたか、または `vc_nfs_disk_size`の値として `0` を設定した場合は、このセクションを実行してもVCディスクは作成されません。

### UnitGroupの作成

VCディスクを管理するための UnitGroup を作成します。


UnitGroupを作成するまえに、現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

UnitGroupを作成します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

if 'disk_unit_group' in gvars and 'vc_nfs_disk_size' in gvars and gvars['vc_nfs_disk_size'] > 0:
    ug_disk = vcp.create_ugroup(gvars['disk_unit_group'], ugroup_type='storage')

UnitGroup作成後の一覧を表示させます。

In [ ]:
vcp.df_ugroups()

### VCディスクの作成

NFS用のVCディスクを作成します。

In [ ]:
if 'ug_disk' in vars() and gvars['vc_nfs_disk_size'] > 0:
    nfs_disk_spec = vcp.get_spec(gvars['vc_provider'] + '_disk', 'small')
    if gvars['vc_provider'] == 'azure':
        nfs_disk_spec.disk_size_gb = gvars['vc_nfs_disk_size']
    elif gvars['vc_provider'] == 'oracle':
        nfs_disk_spec.size_in_gbs = gvars['vc_nfs_disk_size']
    else:
        nfs_disk_spec.size = gvars['vc_nfs_disk_size']
    ug_disk.create_unit('nfs', nfs_disk_spec)

作成したVCディスクの一覧を表示します。

In [ ]:
from IPython.display import display
if 'ug_disk' in vars():
    display(ug_disk.df_nodes())

## VCノードの起動

### managerノード

#### manager用のVCノードを起動する

manager用VCノードの `spec` を指定します。

In [ ]:
import sys
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
spec_mgr = vcp.get_spec(gvars['vc_provider'], gvars['manager_flavor'])

# Baseコンテナイメージを指定する
spec_mgr.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:base-nfsd'
spec_mgr.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

if 'vc_ipaddress' in gvars:
    # manager用VCノードに割り当てるIPアドレスを指定する
    spec_mgr.ip_addresses = [gvars['vc_ipaddress']]
elif 'vc_mac_addresses' in gvars:
    # manager用VCノードに割り当てるMACアドレスを指定する
    spec_mgr.mac_addresses = [gvars['vc_mac_address']]
    
# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_mgr.volume_size = gvars['manager_disk_size']
elif vc_provider == 'azure':
    spec_mgr.disk_size_gb = gvars['manager_disk_size']
else:
    print('This provider does not support the specification of the root volume size.' +
          ' Does not set the disk size.', file=sys.stderr)

# オンプレミスに構築する場合、SSHログインユーザ名を指定
if vc_provider == 'onpremises':
    spec_mgr.user_name = ssh_user

# VCノードにsshでログインするための公開鍵を指定する
spec_mgr.set_ssh_pubkey(gvars['ssh_public_key_path'])
spec_mgr.params_e.append(f"UGROUP_NAME={ugroup_name}")

if 'ug_disk' in vars():
    spec_mgr.disks = ug_disk.find_nodes()
    if len(spec_mgr.disks) > 0:
        spec_mgr.params_e.append("NFS_MKFS=yes")
else:
    spec_mgr.params_v.append('/exported:/exported')

`spec` の設定値を確認します。

In [ ]:
print(spec_mgr)

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(ugroup_name)

manager用VCノードを起動します。

In [ ]:
unit_mgr = ugroup.create_unit('manager', spec_mgr)

起動したVCノードの一覧を表示します。

vcname:　本ノートブックで指定した、`UnitGroup`  
unit_name:　`manager`

In [ ]:
unit_mgr.df_nodes()

VCノードのIPアドレスを変数`vc_ipaddress`に設定します。

In [ ]:
vc_ipaddress = unit_mgr.find_ip_addresses()[0]
print(vc_ipaddress)

`group_vars`ファイルにIPアドレスの値を記録します。

In [ ]:
update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
gvars = load_group_vars(ugroup_name)

#### managerノードに対するAnsibleの設定

起動したVCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in unit_mgr.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
%run scripts/group.py
%run scripts/edit_conf.py

inventory = {'all': {'children': {
    ugroup.name: {
        'children': {
            f'{ugroup.name}_{unit_mgr.name}': {
                'hosts': dict([(x, {}) for x in unit_mgr.find_ip_addresses()]),
            },
        },
        'vars': {
            'ansible_user': 'vcp',
            'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
            'ansible_python_interpreter': '/usr/bin/python3',
        },
    },
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

 先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。
> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

#### NFSサーバの確認

managerノードのNFSサーバのサービス状態を確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'systemctl status nfs-server'

NFSエクスポートの状態を確認します。managerノードの起動時は全てのノードに対してアクセス可能な状態に設定されています。後ほどworkerノードを起動してNFSクライアントのIPアドレスが確定した時点で`/etc/exports`の設定を更新し、アクセスできるノードの制限を行います。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'exportfs -v'

ディレクトリのパーミッションを設定します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -m file -a 'path=/home mode=777'
!ansible {ugroup.name}_{unit_mgr.name} -b -m file -a 'path=/exchange mode=777'

### workerノード

#### worker用のVCノードを起動する

worker用VCノードの `spec` を指定します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
spec_worker = vcp.get_spec(gvars['vc_provider'], gvars['worker_flavor'])

# Baseコンテナイメージを指定する
spec_worker.image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:base'

spec_worker.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# workerノード数を指定する
spec_worker.num_nodes = int(gvars['worker_nodes'])

if 'worker_ipaddresses' in gvars:
    # worker用VCノードに割り当てるIPアドレスを指定する
    spec_worker.ip_addresses = gvars['worker_ipaddresses']
elif 'vc_mac_addresses' in gvars:
    # worker用VCノードに割り当てるMACアドレスを指定する
    spec_worker.mac_addresses = gvars['worker_mac_addresses']

# ルートボリュームサイズを指定する
if vc_provider == 'aws':
    spec_worker.volume_size = gvars['worker_disk_size']
elif vc_provider == 'azure':
    spec_worker.disk_size_gb = gvars['worker_disk_size']
else:
    print('This provider does not support the specification of the root volume size.' +
          ' Does not set the disk size.', file=sys.stderr)

# オンプレミスに構築する場合、SSHログインユーザ名を指定
if vc_provider == 'onpremises':
    spec_worker.user_name = ssh_user

# VCノードにsshでログインするための公開鍵を指定する
spec_worker.set_ssh_pubkey(gvars['ssh_public_key_path'])

# NFSサーバのIPアドレスを指定する
spec_worker.params_e.append("NFS_SERVER=" + gvars['vc_ipaddress'])

`spec` の設定値を確認します。

In [ ]:
print(spec_worker)

worker用VCノードを起動します。

In [ ]:
unit_worker = ugroup.create_unit('worker', spec_worker)

起動したVCノードの一覧を表示します。

vcname:　本ノートブックで指定した、`UnitGroup`  
unit_name:　`worker`

In [ ]:
unit_worker.df_nodes()

#### workerノードに対するAnsibleの設定

VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

In [ ]:
!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in unit_worker.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
inventory = {'all': {'children': {
    ugroup.name: {
        'children': {
            f'{ugroup.name}_{unit_worker.name}': {
                'hosts': dict([(x, {}) for x in unit_worker.find_ip_addresses()]),
            },
        },
    },
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

#### NFSの設定

workerノードにおけるNFSマウントの状態を確認します。`/jupyter`, `/exchange`のエントリが存在していることを確認してください。

In [ ]:
!ansible {ugroup.name}_{unit_worker.name} -a 'mount -t nfs4'

NFSサーバの `/etc/exports` の設定を更新します。NFSサーバにアクセスできるNFSクライアントをworkerノードのみとなるように設定します。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

exports_opts = 'rw,fsid=0,no_root_squash,no_subtree_check,sync,crossmnt'
with TemporaryDirectory() as workdir:
    exports = Path(workdir) / f'{ugroup_name}.exports'
    with exports.open(mode='w') as f:
        for addr in unit_worker.find_ip_addresses():
            f.write(f'/exported/{ugroup_name} {addr}({exports_opts})\n')
    !cat {exports}
    !ansible {ugroup.name}_{unit_mgr.name} -b -m copy -a \
        'src={exports} dest=/etc/exports.d/ backup=yes'

設定ファイルを読み込ませてエクスポート設定を更新します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -b -a 'exportfs -r -v'

NFSサーバ側に作成したファイルをNFSクライアント側で参照できることを確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -m file -a 'path=/jupyter/xxx state=touch'
!ansible {ugroup.name}_{unit_mgr.name} -m file -a 'path=/exchange/xxx state=touch'
!ansible {ugroup.name}_{unit_worker.name} -a 'test -f /jupyter/xxx'
!ansible {ugroup.name}_{unit_worker.name} -a 'test -f /exchange/xxx'
!ansible {ugroup.name} -m file -a 'path=/jupyter/xxx state=absent'
!ansible {ugroup.name} -m file -a 'path=/exchange/xxx state=absent'

## Docker Swarmの設定

Docker Swarm の設定を行います。

Docker Swarmの設定を行う前に、各VCノードで Docker Engine が実行されていることを確認します。

In [ ]:
!ansible {ugroup.name} -a 'docker info'

manager用VCノードで Docker Swarmの初期セットアップを行います。

In [ ]:
manager_ip = gvars['vc_ipaddress']
!ansible {ugroup.name}_{unit_mgr.name} -a 'docker swarm init \
    --default-addr-pool={{{{docker_address_pool}}}} --advertise-addr={manager_ip}'

トークンの値を取得します。

In [ ]:
out = !ansible {ugroup.name}_{unit_mgr.name} -a 'docker swarm join-token -q worker'
if out[0].find('CHANGED') >= 0 or out[0].find('SUCCESS') >= 0:
    swarm_token = out[1]
    print(swarm_token)
else:
    raise

workerノードを追加します。

In [ ]:
!ansible {ugroup.name}_{unit_worker.name} -a \
    'docker swarm join --token {swarm_token} {manager_ip}:2377'

Docker Swarmのノード一覧を表示します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -a 'docker node ls'

Docker Swarmのノード数が起動したVCノードと一致していることを確認します。

In [ ]:
!ansible {ugroup.name}_{unit_mgr.name} -m shell -a \
    'test $(docker node ls -q | wc -l) -eq {worker_nodes + 1}'